## DarkSky API pull for fire post clustered data
### Main columns needed are latitude, longitude, datetime

In [49]:
import pandas as pd
import datetime
import numpy as np

import config

from darksky import forecast
from datetime import datetime as dt
from datetime import timedelta
from datetime import date
from sqlalchemy import create_engine

In [50]:
# create sqlalchemy engine
engine = create_engine("mysql+mysqlconnector://{user}:{password}@{host}/{dbname}"
                       .format(user=config.db_user,
                               password=config.db_pass,
                               dbname=config.db_name,
                               host=config.db_host))

In [19]:
RAPIDAPI_KEY  = config.darksky_api1
RAPIDAPI_KEY2  = config.darksky_api2

In [20]:
def weather_lookup(df, key=RAPIDAPI_KEY, days_before=0, days_after=0):
    data = []
    for index, row in df.iterrows():
        ts = row['datetime'].isoformat() 
        lat = row['latitude']
        lon = row['longitude']
        weather = forecast(key, lat, lon, time=ts)
        w_dict = weather['currently']
        w_dict['timestamp'] = ts
        w_dict['latitude'] = lat
        w_dict['longitude'] = lon
        data.append(w_dict)
    return data


In [21]:
filename = "fire_lat_lon_data/Emissions2008_DBScan_Clusters.csv"

In [22]:
firedata = pd.read_csv(filename)

In [23]:
firedata['datetime'] = (pd.to_datetime(firedata.year, format='%Y') + 
                       firedata['doy'].apply(np.ceil).apply(
                           lambda x: pd.Timedelta(x, unit='D')) + timedelta(hours=12))

In [24]:
firedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9264 entries, 0 to 9263
Data columns (total 24 columns):
Unnamed: 0             9264 non-null int64
id                     9264 non-null float64
year                   9264 non-null float64
doy                    9264 non-null float64
longitude              9264 non-null float64
latitude               9264 non-null float64
grid10k                9264 non-null float64
covertype              9264 non-null float64
fuelcode               9264 non-null float64
area_burned            9264 non-null float64
prefire_fuel           9264 non-null float64
consumed_fuel          9264 non-null float64
ECO2                   9264 non-null float64
ECO                    9264 non-null float64
ECH4                   9264 non-null float64
EPM2.5                 9264 non-null float64
cwd_frac               9264 non-null float64
duff_frac              9264 non-null float64
fuel_moisture_class    9264 non-null float64
burn_source            9264 non-null flo

In [25]:
firedata1000 = firedata[0:1000]

In [26]:
firedata2000 = firedata[1000:2000]

In [27]:
firedata3000 = firedata[2000:3000]

In [28]:
firedata4000 = firedata[3000:4000]

In [29]:
firedata5000 = firedata[4000:5000]

In [32]:
weather_df1 = pd.DataFrame(weather_lookup(firedata1000,RAPIDAPI_KEY))
weather_df2 = pd.DataFrame(weather_lookup(firedata2000,RAPIDAPI_KEY2))

In [59]:
db_columns = ['apparentTemperature', 'cloudCover', 'dewPoint',
       'humidity', 'icon', 'latitude', 'longitude', 'precipAccumulation',
       'precipIntensity', 'precipProbability', 'precipType', 'pressure',
       'summary', 'temperature', 'time', 'timestamp', 'uvIndex', 'visibility',
       'windBearing', 'windGust', 'windSpeed']

In [61]:
weather_df1 = weather_df1[db_columns]

In [63]:
weather_df2 = weather_df2[db_columns]

In [58]:
weather_df1.to_sql('weather_loading', con = engine, if_exists = 'append', chunksize = 100000)

In [ ]:
weather_df2.to_sql('weather_loading', con = engine, if_exists = 'append', chunksize = 100000)